In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# PHYS 395 - week 7

**Matt Wiens - #301294492**

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

*The TA's name (Ignacio) will be shortened to "IC" whenever used.*

## Setup 

In [ ]:
# Set default plot size
plt.rcParams["figure.figsize"] = (12, 9)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Monte Carlo simulation of the Ising model

First we'll define some constants.

In [ ]:
# Aliases
UP = 1
DOWN = -1

In [ ]:
def metropolis(
    num_equilibrium_sweeps: int = 200,
    num_sample_sweeps: int = 50,
    num_samples: int = 300,
):
    pass